# 카운트 기반의 단어 표현(BoW, DTM, TF-IDF)

## [Bag of Words](https://wikidocs.net/22650)

Bag of Words란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법

[BoW를 만드는 과정]
1. 각 단어에 고유한 정수 인덱스를 부여  # 단어 집합 생성.
2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듦.

### step1. CountVectorizer 클래스로 BoW 만들기

#### 문서1: 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.

문서1에 대해 BoW를 만든다. 아래의 함수는 입력된 문서에 대해서 단어 집합을 만들어 각 단어에 정수 인덱스를 할당하고, BoW를 만든다.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지 출력
print('vocabulary :', vector.vocabulary_)

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


you와 love는 두 번씩 언급되었으므로 각각 인덱스 2와 4에서 2의 값을 가지며, 그 외의 값에서는 1의 값을 갖는다.  
또한 알파벳 I는 BoW를 만드는 과정에서 사라졌는데, 이는 CounterVectorizer가 기본적으로 길이가 2 이상인 문자에 대해서만 토큰으로 인식한다.  
영어에서는 길이가 짧은 문자를 제거하는 것도 전처리 작업으로 고려된다.

주의할 것은 **CountVectorizer는 단지 띄어쓰기만을 기준**으로 단어를 자르는 낮은 수준의 토큰화를 진행하고 BoW를 만든다는 점이다. 따라서 한국어에 적용하면 BoW가 제대로 만들어지지 않는다.

### step2. 불용어를 제거한 BoW 만들기

CountVectorizer는 불용어를 지정하면, 불용어는 제외하고 BoW를 만들 수 있도록 불용어 제거 기능을 지원한다.

#### (1) 사용자 정의 불용어 사용

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [18]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=['the', 'a', 'an', 'is', 'not'])
print('bag of words vector: ', vect.fit_transform(text).toarray())
print('vocabulary: ', vect.vocabulary_)

bag of words vector:  [[1 1 1 1 1]]
vocabulary:  {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


#### (2) CountVectorizer에서 제공하는 자체 불용어 사용

In [19]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


#### (3) NLTK에서 지원하는 불용어 사용

In [21]:
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


---
## 문서 단어 행렬(Document-Term Matrix, DTM)

- 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현
- 각 문서에 대한 BoW를 하나의 행렬로 만든 것
- BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어

---
## TF-IDF(Term Frequency-Inverse Document Frequency)

- TF-IDF(Term Frequency-Inverse Document Frequency)는 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰임
- TF-IDF은 TF와 IDF를 곱한 값을 의미
    - **모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단**
    - TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것

tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수.

df(t) : 특정 단어 t가 등장한 문서의 수.

idf(d, t) : df(t)에 반비례하는 수.

In [21]:
### step1. 파이썬으로 TF-IDF 직접 구
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [22]:
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

TF, IDF, 그리고 TF-IDF 값을 구하는 함수를 구현

In [28]:
# 총 문서의 수
N = len(docs) 

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df+1))

def tfidf(t, d):
  return tf(t,d)* idf(t)

TF를 구한다. 다시 말해 DTM을 데이터프레임에 저장하여 출력한다.

In [29]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


각 단어에 대한 IDF 값을 구한다.

In [30]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))
    
idf_=pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


TF-IDF 행렬을 출력한다.

In [31]:
result = []
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### step2. 사이킷런을 이용한 DTM과 TF-IDF 실습

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
  'you know I want your love',
    'I like you',
    'what should I do ',  
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


DTM이 완성됐다. 각 단어의 인덱스가 어떻게 부여되었는지 확인하기 위해 인덱스를 확인해보겠다.  
첫 번째 열의 경우에는 0의 인덱스를 가진 do이다. do는 세번째 문서에만 등장했기 때문에, 세번째 행에서만 1의 값을 가진다. 

사이킷런은 TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공한다.

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'you know I want your love',
    'I like you',
    'what should I do ',  
]

tfidfv = TfidfVectorizer()
print(tfidfv.fit_transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
